#Loading my git repo and  install pakages

In [1]:
!git clone https://github.com/cyrus1123/my-Masters-projects.git

Cloning into 'my-Masters-projects'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


In [2]:
%cd my-Masters-projects 
!unzip MyBERTModel.zip

/content/my-Masters-projects
Archive:  MyBERTModel.zip
   creating: MyBERTModel/
   creating: MyBERTModel/transformers-master/
   creating: MyBERTModel/transformers-master/.circleci/
  inflating: MyBERTModel/transformers-master/.circleci/config.yml  
  inflating: MyBERTModel/transformers-master/.circleci/deploy.sh  
  inflating: MyBERTModel/transformers-master/.coveragerc  
  inflating: MyBERTModel/transformers-master/.gitattributes  
   creating: MyBERTModel/transformers-master/.github/
   creating: MyBERTModel/transformers-master/.github/conda/
  inflating: MyBERTModel/transformers-master/.github/conda/build.sh  
  inflating: MyBERTModel/transformers-master/.github/conda/meta.yaml  
   creating: MyBERTModel/transformers-master/.github/ISSUE_TEMPLATE/
  inflating: MyBERTModel/transformers-master/.github/ISSUE_TEMPLATE/---new-benchmark.md  
  inflating: MyBERTModel/transformers-master/.github/ISSUE_TEMPLATE/--new-model-addition.md  
  inflating: MyBERTModel/transformers-master/.github/

In [3]:
!pip install transformers==3.0.2
!pip install joblib
!pip install sentencepiece
!pip install urllib3
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask

     |████████████████████████████████| 778kB 5.7MB/s 
     |████████████████████████████████| 901kB 7.5MB/s 
     |████████████████████████████████| 1.2MB 18.7MB/s 
     |████████████████████████████████| 3.0MB 25.8MB/s 


In [4]:
%cd MyBERTModel


/content/my-Masters-projects/MyBERTModel


#Loading  GUI

In [ ]:
# Run this block and wait till it gerenerates 3 links.
#Then, click on the link that contains ngrok.io
import os
from flask import Flask, render_template
from flask import request
from flask_ngrok import run_with_ngrok
from flask import Flask
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# name = "twmkn9/albert-base-v2-squad2"
# name = "distilbert-base-uncased-distilled-squad"
name = "mrm8488/bert-small-finetuned-squadv2"

tokenizer = AutoTokenizer.from_pretrained(name,)

model = AutoModelForQuestionAnswering.from_pretrained(name)

def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer` string and tries to identify 
    the words within the `answer` that can answer the question. Prints them out.
    '''
    
    # tokenize the input text and get the corresponding indices
    token_indices = tokenizer.encode(question, answer_text)

    # Search the input_indices for the first instance of the `[SEP]` token.
    sep_index = token_indices.index(tokenizer.sep_token_id)

    seg_one = sep_index + 1

    # The remainders lie in the second segment.
    seg_two = len(token_indices) - seg_one
    
    # Construct the list of 0s and 1s.
    segment_ids = [0]*seg_one + [1]*seg_two

    # get the answer for the question
    start_scores, end_scores = model(torch.tensor([token_indices]), # The tokens representing our input combining question and answer.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer

    # Find the tokens with the highest `start` and `end` scores.
    answer_begin = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    indices_tokens = tokenizer.convert_ids_to_tokens(token_indices)
    
    answer = indices_tokens[answer_begin:answer_end+1]
    #remove special tokens
    answer = [word.replace("▁","") if word.startswith("▁") else word for word in answer] #use this when using model "twmkn9/albert-base-v2-squad2"
    answer = " ".join(answer).replace("[CLS]","").replace("[SEP]","").replace(" ##","")
    
    return answer


app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run


@app.route('/', methods=['GET', 'POST'])
def index():
  
    if request.method == 'POST':
      form = request.form
      result = []
      bert_abstract = form['paragraph']
      question = form['question']
      result.append(form['question'])
      result.append(answer_question(question, bert_abstract))
      result.append(form['paragraph'])

      return render_template("index.html",result = result)

    # answer = answer_question(question, bert_abstract)

    # return answer
    return render_template("index.html")

if __name__ == '__main__':
    
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7977b8645900.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [30/Apr/2021 05:34:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Apr/2021 05:34:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [30/Apr/2021 05:34:50] "GET / HTTP/1.1" 200 -
